# DECISION STRUCTURE MASS

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import ipywidgets as ipw
import IPython.display as ipd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as plot
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff

In [2]:
#Scorecard results
def _scorecard (score):
    if -2.00 <= score < -1.25: score = 0
    elif -1.25 <= score < -0.75: score = 1
    elif -0.75 <= score < -0.25: score = 2
    elif -0.25 <= score <= 0.25: score = 3
    elif 0.25 < score <= 0.75: score = 4
    elif 0.75 < score <= 1.25: score = 5
    elif 1.25 < score <= 2.00: score = 6    
    else: score = 7
    switcher = {
        0: [-3, 'UW_Max'],
        1: [-2, 'UW'],
        2: [-1, 'UW_Slight'],
        3: [0, 'Neutral'],
        4: [1, 'OW_Slight'],
        5: [2,'OW'],
        6: [3, 'OW_Max']}    
    return switcher.get(score, "Invalid")

def _tier_grp (score):
    if score == 1: score = 0
    elif score == 2: score = 1
    elif score == 3: score = 2
    elif score == 4: score = 3
    elif score == 5: score = 4
    switcher = {
        0:'Tier_A',
        1:'Tier_B',
        2:'Tier_C',
        3:'Tier_D',
        4:'Tier_E'}    
    return switcher.get(score, "Invalid")

In [3]:
#random input
#import random
#random.seed(0)

def _gen_rand_MASS_input():

    random_input = []
    total_alloc = 0
    while total_alloc < 1.0:
        saa_wgt = np.random.uniform(0.01,0.20)
        ltaa_wgt = np.random.uniform(0.01,saa_wgt)
        utaa_wgt = np.random.uniform(saa_wgt,0.80)
        score = np.random.uniform(-2,2)
        tier = np.random.choice([1,2,3,4,5])
    
        total_alloc += saa_wgt

        random_input.append([saa_wgt,score,utaa_wgt,ltaa_wgt,float(tier)])

    name = []
    for i in range(len(random_input)):
        name.append('asset_'+str(i))
    
    rnd = pd.DataFrame(random_input,index=name,columns=['port_alloc','class_score','up_rng','low_rng','port_prio'])

    wgt = []
    low = []
    for index, row in rnd.iterrows():
        total = rnd['port_alloc'].sum()
        re_alloc = rnd['port_alloc'].sum() - 1
        _wgt = row['port_alloc'] - (row['port_alloc']/total*re_alloc)
        _low = row['low_rng'] - (row['port_alloc']/total*re_alloc)
    
        wgt.append(_wgt)
    
        if _low < 0 :
            low.append(0)
        else:
            low.append(_low)
    
    rnd['port_alloc'] = wgt
    rnd['low_rng'] = low

    cash_wgt = np.random.uniform(0.01,0.20) #1-rnd['low_rng'].sum()
    cash_ltaa_wgt = np.random.uniform(0.01,cash_wgt)
    cash_utaa_wgt = np.random.uniform(cash_wgt,0.20) #1-rnd['low_rng'].sum()

    final_wgt = []
    final_low = []
    for index, row in rnd.iterrows():
        total = rnd['port_alloc'].sum()
        re_alloc = cash_wgt
        _wgt = row['port_alloc'] - (row['port_alloc']/total*re_alloc)
        _low = row['low_rng'] - (row['port_alloc']/total*re_alloc)
    
        final_wgt.append(_wgt)
    
        if _low < 0 :
            final_low.append(0)
        else:
            final_low.append(_low)

    rnd['port_alloc'] = final_wgt
    rnd['low_rng'] = final_low

    rnd = rnd.transpose().round(4)

    sample_cash = pd.Series([cash_wgt,np.nan,cash_utaa_wgt,cash_ltaa_wgt,np.nan],name='CASH',index=['port_alloc','class_score','up_rng','low_rng','port_prio'])

    rnd = pd.concat([rnd,sample_cash],axis=1).transpose().round(4)
    #sample_cash

    return rnd

#rnd = _gen_rand_MASS_input()
#rnd

In [4]:
#unrestrained weight

def _un_wgt(rnd):

    feature = rnd.values[:-1] #ex cash
    saa = feature[:,0] #saa
    score = feature[:,1] #score
    urng = feature[:,2] #urng
    lrng = feature[:,3] #lrng
    prio = feature[:,4] #prio

    tilt = []
    for i in score:
        tilt.append(_scorecard(i)[0])
    
    #print(tilt)
    
    counter = 0
    app_tilt = np.array([])
    for i in tilt:
        #print(type(i))
        if i > 0:
            t = saa[counter] + ((urng[counter]-saa[counter])/3*abs(i))
            app_tilt = np.append(app_tilt,t)
        elif i < 0:
            t = saa[counter] - ((saa[counter]-lrng[counter])/3*abs(i))
            app_tilt = np.append(app_tilt,t)
        else:
            app_tilt = np.append(app_tilt,saa[counter])
        
        counter += 1
        
    feature = np.append(feature,app_tilt.reshape(-1,1),axis=1)
    feature = np.append(feature,app_tilt.reshape(-1,1),axis=1) # added extra column
    
    return feature

#feature = _un_wgt(rnd)
#feature#[:,5]
#feature

In [5]:
#available gap

def _available_gap(feature):

    un_wgt = feature[:,5]
    cash = rnd.values[-1]
    cash_saa = cash[0]
    cash_urng = cash[2]
    cash_lrng = cash[3]
    
    saa = feature[:,0] #saa
    score = feature[:,1] #score
    urng = feature[:,2] #urng
    lrng = feature[:,3] #lrng
    prio = feature[:,4] #prio
    
    wgt_bal = {}
    gap = {}
    tier = 5 #len(np.unique(prio))
    for i in range(1,tier+1):
        wgt_bal[i] = 0.00
        gap[i] = 0.00

    def avail_gap():
        gap = 0
        if un_wgt.sum() + cash_urng < 1: #<-- cash balance included for condition check only
            gap += urng[counter] - un_wgt[counter]
        elif un_wgt.sum() + cash_lrng > 1: #<-- cash balance included for condition check only
            gap += un_wgt[counter] - lrng[counter]
        else: gap = 0
        return gap
    
    counter = 0
    for i in prio:
        if i == 1:
            wgt_bal[1]+=un_wgt[counter]
            gap[1] = 0 #avail_gap() #<-- Tier A absolute
        elif i == 2:
            wgt_bal[2]+=un_wgt[counter]
            gap[2] += avail_gap()
        elif i == 3:
            wgt_bal[3]+=un_wgt[counter]
            gap[3] += avail_gap()
        elif i == 4:
            wgt_bal[4]+=un_wgt[counter]
            gap[4] += avail_gap()
        elif i == 5:
            wgt_bal[5]+=un_wgt[counter]
            gap[5] += avail_gap()
        counter+=1
        
    bucket = np.array(list(wgt_bal.values())) #<-- cash balance not included
    gap_bucket = np.array(list(gap.values())) #<-- cash balance not included

    #bucket = bucket[::-1]
    #gap_bucket = gap_bucket[::-1]
    
    #print(wgt_bal)
    #print(rnd)
    return bucket, gap_bucket

#b,g = _available_gap(feature)
#print(b)
#print(g)

In [6]:
#tier priority

def asset_dict(rnd,feature):
    sec_level_dict = {}
    counter = 0
    for i in rnd.index[:-1]:
        sec_level_dict[i] = feature[counter]
        counter += 1
    return sec_level_dict

def tier_dict(dct,num_range):
    def sort_dict(i,dct):
        tier_dict = {}
        for k,v in dct.items():
            if v[4] == i:
                tier_dict[k] = v
        return tier_dict
    
    order_dict = {}
    order = list(range(1,num_range+1))[::-1] #<-- review reversed
    for i in order:
        order_dict[i] = sort_dict(i,dct)    
    
    return order_dict

#order_dict = tier_dict(asset_dict(rnd,feature),5)
#order_dict
#pd.DataFrame(asset_dict(rnd,feature)).transpose()

In [7]:
#rebalancer

def sec_level_wgt(dct,num_range): 
    
    order = list(range(1,num_range+1))[::-1] #<-- review reversed
    
    cash = rnd.values[-1]
    cash_saa = cash[0]
    cash_urng = cash[2]
    cash_lrng = cash[3]
    
    balance = 1-bucket.sum() #<-- not adjusted for cash spread
    
    
    def _excess(balance):
        
        def _excess_rebalancer(dct,tier,_bal):
            
            gb_index = tier-1 
            check_gap = 0
            roll_bal = 0
            
            for k,v in dct[tier].items():
                if gap_bucket[gb_index]>_bal:
                    check_gap = _bal
                else:
                    check_gap = gap_bucket[gb_index] 
                
                sec_level_gap = v[5] - v[3]
                sec_level_increment = (check_gap * (sec_level_gap/gap_bucket[gb_index]))
                np.seterr('ignore')
                sec_level_alloc = v[-1] - sec_level_increment
                
                if np.isnan(sec_level_alloc) or np.isinf(sec_level_alloc):
                    pass
                else:
                    v[-1] = sec_level_alloc
                
                #print(k,v[-1].round(4),gap_bucket[gb_index].round(4),sec_level_gap.round(4),sec_level_alloc.round(4))
                
                roll_bal += sec_level_increment
                
            return roll_bal
        
        roll_bal = balance
        for i in order:
            roll_bal -= _excess_rebalancer(order_dict,i,roll_bal)
            
            #print(roll_bal)
            
        return order_dict
    
    
    def _residual(balance):
        
        def _residual_rebalancer(dct,tier,_bal):
        
            gb_index = tier-1 
            check_gap = 0
            roll_bal = 0
    
            for k,v in dct[tier].items():
                if gap_bucket[gb_index]>_bal:
                    check_gap = _bal
                else:
                    check_gap = gap_bucket[gb_index] 
    
                sec_level_gap = v[2] - v[5]
                sec_level_increment = (check_gap * (sec_level_gap/gap_bucket[gb_index]))
                np.seterr('ignore')
                sec_level_alloc = v[-1] + sec_level_increment
                
                #print('print: ',check_gap,_bal,gap_bucket[gb_index])
                
                if np.isnan(sec_level_alloc) or np.isinf(sec_level_alloc):
                    pass
                else:
                    v[-1] = sec_level_alloc
                
                #print('print: ',k,v[-1].round(4),gap_bucket[gb_index].round(4),sec_level_gap.round(4),sec_level_alloc.round(4))
                
                roll_bal += sec_level_increment
                
            return roll_bal
    
        roll_bal = balance
        for i in order:
            roll_bal -= _residual_rebalancer(order_dict,i,roll_bal)
            
            #print(roll_bal)
            
        return order_dict
    
    #residual/excess checker
    _res = 1 - (bucket.sum() + cash_urng)
    _exc = (bucket.sum() + cash_lrng) - 1
    
    wgt_low = 1 - cash_urng
    wgt_high = 1 - cash_lrng
    
    if wgt_low <= bucket.sum() <= wgt_high:
        final_cash = 1 - bucket.sum()
        
        print('Final weight is within range: ', final_cash )
        
        # append cash specs for the final weight
        
        return order_dict
        
    
    elif 0 < _res < 1: #bucket.sum() + cash_urng
        balance = 1 - (bucket.sum() + cash_urng)
        print(balance,' residual')
        
        # insert post alloc checker
        
        return _residual(balance)# - cash_urng)
    
    elif _exc > 0: #bucket.sum() + cash_lrng #balance < 0:
        balance = (bucket.sum() + cash_lrng) - 1 #abs(balance)
        print(balance,' excess')
        
        # insert post alloc checker
        
        return _excess(balance) # + cash_lrng)
    
    elif _res == 0 and _exc == 0:
        print('Initial allocation within limits')
        return order_dict
    
    else: #pass #return order_dict
        print('recheck excess/residual balances')
        print('resid: ', 1 - (bucket.sum() + cash_urng))
        print('excess: ', (bucket.sum() + cash_lrng) - 1)
        
        return order_dict
    
#final_order_dict = sec_level_wgt(order_dict,5)
#final_order_dict
#asset, values, gap_bucket, sec_level_gap, sec_level_alloc

In [8]:
#assertion functions

#pre allocation functions
#input checker

def _input_data_checker(df):
    
    cash_net = rnd.iloc[:-1,:]
    cash_spec = rnd.values[-1]
    max_cash = cash_spec[2]
    min_taa = cash_net['low_rng'].sum()
    
    if df['port_alloc'].sum().round(2) != 1:
        print('Check SAA Total: ',df['port_alloc'].sum().round(2))
        raise Exception('Strategic Asset Allocation must equal 100')
    
    elif df['port_alloc'].sum().round(2) > 1:
        print('Total Strategic Asset Allocation > 100%')
        raise Exception('Total Strategic Asset Allocation > 100%')
    
    elif df['port_alloc'].sum() == 0:
        print('No Input Data Detected')
        raise Exception('No Input Data Detected')

    elif max_cash > (1-min_taa).round(4): 
        print('Adjust Cash Levels - Max Cash exposure > (100 - Total of Minimum TAAs)')
        raise Exception('Error: Adjust Cash Maximum TAA')
    
    for index, row in df.iterrows():
        if row['up_rng'] < row['port_alloc'] or row['low_rng'] > row['port_alloc']:
            print(row.name, ' Check Max and Min')
            raise Exception('Check Max and Min')
            
    else: print('Input data conditions met')
    
    return None
            
            

#post allocation functions

def _balance_check():
    port_sum = 0
    for k,v in final_order_dict.items():
        for x,y in v.items():
            port_sum += y[-1]
            print(x,y[4],y[-1].round(4)*100)
    return port_sum.round(4)

def _formatter():
    asset_list = []
    value_list = []
    for k,v in final_order_dict.items():
        for x,y in v.items():
            asset_list.append(x)
            value_list.append(y)
    df = pd.DataFrame(value_list,index=[asset_list],
                      columns=['saa','score','urng','lrng',
                               'prio','init_wgt','final_wgt']).sort_values(by='prio',ascending=True)
    return df


def _cat_tier_check(df):
    category_list =['UW_Max','UW','UW_Slight','Neutral','OW_Slight','OW','OW_Max']
    tier_list = ['Tier_A','Tier_B','Tier_C','Tier_D','Tier_E']
    
    _cat = []
    _tier = []
    for index, row in df.iterrows():
        _cat.append(_scorecard(row['score'])[1])
        _tier.append(_tier_grp(row['prio']))
        
    df['_cat'] = _cat
    df['_tier'] = _tier
    
    for cat in category_list:    
        if df['_cat'].eq(cat).all():
            for tier in tier_list:
                if df['_tier'].eq(tier).all(): 
                    df['final_wgt'] = df['saa'] 
                    print('All categories and tiers are equal, revert to SAA')
    return df


def _tierA_check(df):
    if df[df['prio'] == 1]['init_wgt'].sum() > 1:
        raise Exception("Error: Re-classify Tier A, Final holdings of Tier A assets exceeds 100%")
    return df



def _final_assertion():
    
    checks = _formatter().sum()

    chk_fwgt = checks[-1].round(2)
    chk_ex = (1 - cash_lrng).round(2)
    chk_res = (chk_fwgt + cash_urng).round(2)

    if chk_fwgt > chk_ex:
        print("Warning: Excess holdings needs to be reduced, re-adjust tiers, cash, and/or TAA low range")
    elif chk_res < 1:
        print("Warning: Residual holdings needs to be bought, re-adjust tiers, cash, and/or TAA upper range")
    
    else: print('All conditions met')
        # within limit range instance

    print(checks[-2:])
    print('Cash: ',cash)
    
    #print(chk_fwgt)
    #print(chk_ex)
    #print(chk_res)
    
    return None


def _viz_output(df):
    
    return df

In [9]:
#input function
rnd = _gen_rand_MASS_input()
_input_data_checker(rnd)
cash = rnd.values[-1]
cash_saa = cash[0]
cash_urng = cash[2]
cash_lrng = cash[3]
#rnd

#function map

feature = _un_wgt(rnd)

#print(pd.DataFrame(feature))
bucket, gap_bucket = _available_gap(feature)

order_dict = tier_dict(asset_dict(rnd,feature),5)

final_order_dict = sec_level_wgt(order_dict,5)


#print(_balance_check())
#final_order_dict

#df = _cat_tier_check(_formatter())

_test_output = _tierA_check(_cat_tier_check(_formatter()))
_test_output['saa'] = _test_output['saa'] * 100
_test_output['urng'] = _test_output['urng'] * 100
_test_output['lrng'] = _test_output['lrng'] * 100
_test_output['init_wgt'] = _test_output['init_wgt'] * 100
_test_output['final_wgt'] = _test_output['final_wgt'] * 100
_test_output['delta'] = _test_output['final_wgt'] - _test_output['init_wgt']

_final_assertion()
_test_output.round(2)

Input data conditions met
0.4696  excess
All conditions met
init_wgt     1.4576
final_wgt    0.9880
dtype: float64
Cash:  [0.1       nan 0.1301 0.012     nan]


,saa,score,urng,lrng,prio,init_wgt,final_wgt,_cat,_tier,delta
asset_3,14.86,1.72,28.00,2.73,1.0,28.00,28.00,OW_Max,Tier_A,0.00
asset_5,6.93,0.65,31.11,5.48,1.0,14.99,14.99,OW_Slight,Tier_A,0.00
asset_8,16.33,1.05,23.01,14.38,2.0,20.78,19.47,OW,Tier_B,-1.31
asset_2,7.45,0.75,31.97,6.65,3.0,23.80,6.65,OW,Tier_C,-17.15
asset_4,1.66,-0.97,28.05,1.63,4.0,1.64,1.63,UW,Tier_D,-0.01
asset_7,0.93,-0.85,56.94,0.90,4.0,0.91,0.90,UW,Tier_D,-0.01
asset_0,5.39,-0.10,53.79,4.98,5.0,5.39,4.98,Neutral,Tier_E,-0.41
asset_1,10.90,0.48,74.41,9.19,5.0,32.07,9.19,OW_Slight,Tier_E,-22.88
asset_6,11.88,-1.63,61.68,4.52,5.0,4.52,4.52,UW_Max,Tier_E,-0.00
asset_9,13.66,-0.09,65.45,8.47,5.0,13.66,8.47,Neutral,Tier_E,-5.19


In [10]:
#_test_output.sum()
#_formatter().sum()

In [11]:
wgt_bal = {}
gap = {}
tier = 5 

for i in range(1,tier+1):
    wgt_bal[i] = 0.00
    gap[i] = 0.00
    
list1 = [1,2,3,4,5,5,5,5]
len(np.unique(list1)) == max(list1)



for i in range(0,10):
    gap[5]+=1
    
gap[5]


10.0

In [12]:
list(range(1,6))[::-1]
test_list = [5,4,3,2,1]
test_list[::-1]

[1, 2, 3, 4, 5]

In [13]:
pick = np.array([0,1,2,3,4])
pick[-1] = 5
pick

array([0, 1, 2, 3, 5])

In [14]:
#test_sort = test_sort[test_sort[:,4].argsort()]
arr = [[3,2,5],[5,4,8]]
arr = np.sort(arr,axis=-1)
arr

array([[2, 3, 5],
       [4, 5, 8]])

In [15]:
numbers = [1, 2, 3]
letters = ['a', 'b', 'c']
zipped = zip(numbers, letters)
list(zipped)

[(1, 'a'), (2, 'b'), (3, 'c')]

In [16]:
def get_inputs(x_dict):
    inputs_dict = {}
    for key,widget in x_dict.items():
        if hasattr(widget,'value'):
            inputs_dict[key] = widget.value
    return inputs_dict

In [17]:
def format_display_viz():
    format_display_viz = tier_final_alloc()
    print('Final Total: ',format_display_viz['tier_alloc_final'].sum()*100)
    format_display_viz['port_alloc'] = format_display_viz['port_alloc']*100
    format_display_viz['up_rng'] = format_display_viz['up_rng']*100
    format_display_viz['low_rng'] = format_display_viz['low_rng']*100
    format_display_viz['tier_alloc_prop'] = format_display_viz['tier_alloc_prop']*100
    format_display_viz['tier_alloc_final'] = format_display_viz['tier_alloc_final']*100
    format_display_viz.rename(columns={'port_alloc':'Strategic Asset %',
                                   'up_rng':'Max Tactical %',
                                   'low_rng':'Min Tactical %',
                                   'score_category':'Score Category',
                                   'tier_category':'Tier Bucket',
                                   'tier_alloc_prop':'Unrestrained %',
                                   'tier_alloc_final':'Final Asset %',},inplace=True)
    format_titles = ['Strategic Asset %','Min Tactical %','Max Tactical %','Score Category',
                     'Tier Bucket','Unrestrained %','Final Asset %']
    format_display_viz = format_display_viz.reindex(columns=format_titles)
    
    return format_display_viz


#tier_alloc()
#tier_final_alloc()
#format_display_viz()

In [20]:
#UI    
def _init_UI ():
    
    asset_list = list(test_src.index)[:-1]
    
    #testing = len(asset_list)
    
    #for i in range(0,testing):
        #asset_list.append('test_asset_'+str(i))
        
    layout = ipw.Layout(width ='150px')
    layout_p = ipw.Layout(width ='175px')
    options = [('Tier A',1),('Tier B',2),('Tier C',3),('Tier D',4),('Tier E',5)]
    value = 2
    score_min = -2
    score_max = 2
    step = 0.01
    
    
    heading = ipw.HTML('''<b><h18> Multi-Asset Scoring System </h18></b>''')
    saa = ipw.HTML('''<b><h18> SAA Weight </h18></b>''',layout=layout)
    tilt = ipw.HTML('''<b><h18> Tilt Score </h18></b>''',layout=layout)
    max_taa = ipw.HTML('''<b><h18> Max TAA </h18></b>''',layout=layout)
    min_taa = ipw.HTML('''<b><h18> Min TAA </h18></b>''',layout=layout)
    tier = ipw.HTML('''<b><h18> Tier Category </h18></b>''',layout=layout)
    
    
    title = ipw.HBox([heading])
    space = ipw.HBox([],layout=ipw.Layout(height='20px'))
    
    calc = ipw.Button(description="Calculate")
    clear = ipw.Button(description="Clear")
    refresh = ipw.Button(description="Refresh")
    
    output = ipw.Output()

    def calc_button(b):
        with output:
            output.clear_output()
            format_display_check()
            display(format_display_viz())
            
    def clear_button(b):
        with output:
            for i in _saa_list: i.value = 0
            for i in _min_taa: i.value = 0
            for i in _max_taa: i.value = 0
            for i in _score: i.value = 0
            for i in _prio: i.value = 2
            CLASS_CASH.value = 0
            uCLASS_CASH.value = 0
            lCLASS_CASH.value = 0
            output.clear_output()    
    
    def refresh_button(b):
        with output:
            counter = 0 
            for i in _saa_list: 
                i.value = test_src['port_alloc'][counter]*100
                counter += 1
    
            counter = 0 
            for i in _min_taa:
                i.value = test_src['low_rng'][counter]*100
                counter += 1
    
            counter = 0 
            for i in _max_taa:
                i.value = test_src['up_rng'][counter]*100
                counter += 1

            counter = 0 
            for i in _score:
                i.value = test_src['class_score'][counter]
                counter += 1

            counter = 0 
            for i in _prio:
                i.value = test_src['port_prio'][counter]
                counter += 1
        
            CLASS_CASH.value = 0
            uCLASS_CASH.value = 0
            lCLASS_CASH.value = 0
    
    
    calc.on_click(calc_button)
    clear.on_click(clear_button)
    refresh.on_click(refresh_button)
    
    
    CLASS_CASH = ipw.BoundedFloatText(min=0, style={'description_width':'initial'},layout=layout,description='Cash SAA')
    uCLASS_CASH = ipw.BoundedFloatText(min=0, style={'description_width':'initial'},layout=layout,description='MAX Cash')
    lCLASS_CASH = ipw.BoundedFloatText(min=0, style={'description_width':'initial'},layout=layout,description='MIN Cash')
    
    asset_grp = ipw.HBox([])
    asset_container = ipw.VBox([])
    
    weight_dict = {}
    l_rng_dict = {}
    u_rng_dict = {}
    score_dict = {}
    prio_dict = {}
    
    _saa_list = []
    _min_taa = []
    _max_taa = []
    _score = []
    _prio = []

    for n in asset_list:
        asset_saa = ipw.BoundedFloatText(min=0,description=n,layout=layout)
        minimum = ipw.BoundedFloatText(min=0,description='Min',layout=layout)
        maximum = ipw.BoundedFloatText(min=0,description='Max',layout=layout)
        score = ipw.BoundedFloatText(min=score_min,max=score_max,step=step,description='Score',layout=layout)
        prio = ipw.Dropdown(options=options,value=value,description=n,layout=layout_p)
    
        _saa_list.append(asset_saa)
        _min_taa.append(minimum)
        _max_taa.append(maximum)
        _score.append(score)
        _prio.append(prio)
    
        asset_grp = ipw.HBox([asset_saa,minimum,maximum,score,prio])
        asset_container = ipw.VBox([asset_container, asset_grp])
        
        weight_dict[n] = asset_saa
        l_rng_dict[n] = minimum
        u_rng_dict[n] = maximum
        score_dict[n] = score
        prio_dict[n] = prio
        

    cash_cont = ipw.HBox([CLASS_CASH, space, lCLASS_CASH, uCLASS_CASH])
    
    buttons = ipw.HBox([calc,clear,refresh]) 
    
    sub_head = ipw.HBox([saa,min_taa,max_taa,tilt,tier])
    
    container = ipw.VBox([title, space, sub_head, asset_container, space, cash_cont, space, buttons, output])
    
    weight_dict['CASH'] = CLASS_CASH
    u_rng_dict['CASH'] = uCLASS_CASH
    l_rng_dict['CASH'] = lCLASS_CASH
    
    
    #final_df = pd.DataFrame([weight_dict,u_rng_dict,l_rng_dict,score_dict,prio_dict])
    
    return container, weight_dict, score_dict, u_rng_dict, l_rng_dict, prio_dict

ui, weight_dict, score_dict, u_rng_dict, l_rng_dict, prio_dict = _init_UI()

In [21]:
ui

In [19]:
test_src = pd.read_csv('test_src.csv')
test_src.set_index('id', inplace=True)
test_src.info()

<class 'pandas.core.frame.DataFrame'>
Index: 57 entries, BDO PM EQUITY to Cash
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   port_alloc   57 non-null     float64
 1   class_score  56 non-null     float64
 2   up_rng       57 non-null     float64
 3   low_rng      57 non-null     float64
 4   port_prio    56 non-null     float64
dtypes: float64(5)
memory usage: 2.7+ KB


In [ ]:
#input function
rnd = test_src#_gen_rand_MASS_input()
_input_data_checker(rnd)
cash = rnd.values[-1]
cash_saa = cash[0]
cash_urng = cash[2]
cash_lrng = cash[3]

feature = _un_wgt(rnd)
bucket, gap_bucket = _available_gap(feature)
order_dict = tier_dict(asset_dict(rnd,feature),5)
final_order_dict = sec_level_wgt(order_dict,5)

_test_output = _tierA_check(_cat_tier_check(_formatter()))
_test_output['saa'] = _test_output['saa'] * 100
_test_output['urng'] = _test_output['urng'] * 100
_test_output['lrng'] = _test_output['lrng'] * 100
_test_output['init_wgt'] = _test_output['init_wgt'] * 100
_test_output['final_wgt'] = _test_output['final_wgt'] * 100
_test_output['delta'] = _test_output['final_wgt'] - _test_output['init_wgt']

_final_assertion()
_test_output.round(2)

In [ ]:
_test_output.sum()